In [16]:
!pip install git+https://github.com/GJG1203/RobustConAcq.git
!pip install networkx

  Cloning https://github.com/GJG1203/RobustConAcq.git to c:\users\gertj\appdata\local\temp\pip-req-build-ilgbx876
  Resolved https://github.com/GJG1203/RobustConAcq.git to commit e922cca5b082700b94d648f9b0e27456d6d1adb7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/GJG1203/RobustConAcq.git 'C:\Users\gertj\AppData\Local\Temp\pip-req-build-ilgbx876'


In [17]:
print('import succeeded')

import succeeded


In [18]:
from pycona import *

In [19]:
import cpmpy as cp

def construct_nurse_rostering(shifts_per_day, num_days, num_nurses, nurses_per_shift):

    # Define the variables
    roster_matrix = cp.intvar(1, num_nurses, shape=(num_days, shifts_per_day, nurses_per_shift), 
                              name="shifts")


    # Create the language:
    AV = absvar(2)  # create abstract vars - as many as maximum arity
    # create abstract relations using the abstract vars
    lang = [AV[0] == AV[1], AV[0] != AV[1], AV[0] < AV[1], AV[0] > AV[1], AV[0] >= AV[1], AV[0] <= AV[1]]

    instance = ProblemInstance(variables=roster_matrix, language=lang)

    return instance

In [20]:
instance = construct_nurse_rostering(3, 2, 8, 2)
print(instance)

ProblemInstance: 

Variables: [[[shifts[0,0,0] shifts[0,0,1]]
  [shifts[0,1,0] shifts[0,1,1]]
  [shifts[0,2,0] shifts[0,2,1]]]

 [[shifts[1,0,0] shifts[1,0,1]]
  [shifts[1,1,0] shifts[1,1,1]]
  [shifts[1,2,0] shifts[1,2,1]]]].

Language: [(AV4) == (AV5), (AV4) != (AV5), (AV4) < (AV5), (AV4) > (AV5), (AV4) >= (AV5), (AV4) <= (AV5)].


In [21]:
def construct_nurse_rostering_with_oracle(shifts_per_day, num_days, num_nurses, nurses_per_shift):
    
    inst = construct_nurse_rostering(shifts_per_day, num_days, num_nurses, nurses_per_shift)
    
    # Create an oracle for simulation, by defining the ground truth problem
    ## Define the variables
    roster_matrix = inst.variables
    
    ## Define the list of (fixed-arity/decomposed) constraints
    C_T = []

    ## Constraint: Each shift in a day must be assigned to a different nurse
    for day in range(num_days):
        C_T += cp.AllDifferent(roster_matrix[day, ...]).decompose()   # ... means all remaining dimensions

    ## Constraint: The last shift of a day cannot have the same nurse as the first shift of the next day
    for day in range(num_days - 1):
        C_T += cp.AllDifferent(roster_matrix[day, shifts_per_day - 1], roster_matrix[day + 1, 0]).decompose()

    oracle = ConstraintOracle(C_T)

    return inst, oracle

In [22]:
instance, oracle = construct_nurse_rostering_with_oracle(3, 2, 8, 2)

ca = RobustAcq()
learned_instance = ca.learn(instance, oracle=oracle, verbose=1)

NameError: name 'RobustAcq' is not defined